## Tensorflow + RNN 을 활용한 영화 리뷰 감성 분석

#### (1) 전처리 된 데이터 로드

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


#### (2) 토크나이저 함수 정의

In [0]:
! wget --no-check-certificate https://raw.github.com/SOMJANG/Mecab-ko-for-Google-Colab/master/install_mecab-ko_on_colab190912.sh
!bash install_mecab-ko_on_colab190912.sh

--2019-11-12 19:57:53--  https://raw.github.com/SOMJANG/Mecab-ko-for-Google-Colab/master/install_mecab-ko_on_colab190912.sh
Resolving raw.github.com (raw.github.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.github.com (raw.github.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/SOMJANG/Mecab-ko-for-Google-Colab/master/install_mecab-ko_on_colab190912.sh [following]
--2019-11-12 19:57:54--  https://raw.githubusercontent.com/SOMJANG/Mecab-ko-for-Google-Colab/master/install_mecab-ko_on_colab190912.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2155 (2.1K) [text/plain]
Saving to: ‘install_mecab-ko_on_colab190912.sh’

install_mecab-ko_on 10

In [0]:
import pickle

with open("/content/drive/My Drive/Colab Notebooks/preprocessed_data.pkl", "rb") as f:
  saved_data = pickle.load(f)
  
word2idx = saved_data["word2idx"]

In [0]:
import numpy as np
from konlpy.tag import Mecab
mecab = Mecab()

def tokenizer(sent):
  res = []
  tmp = mecab.pos(sent)
  tmp = list(map(lambda x: x[0]+"/"+x[1], tmp))
  for i, word in enumerate(tmp):
    idx = word2idx.get(word)
    if idx != None:
      res.append(idx)
    else:
      res.append(word2idx.get("<UNK>")) # out of vocab word 처리

  return np.array([res], dtype="int32")

#### (3) 학습된 모델 로드

In [0]:
import tensorflow as tf

model = tf.keras.models.load_model("/content/drive/My Drive/Colab Notebooks/best_model.h5")
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         6125100   
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         84992     
___

#### (4) 직접 만든 문장에 대한 추론 수행

In [0]:
test_sample = "이 영화 너무 재밌어요!!! 강추"
model.predict(tokenizer(test_sample))

array([[0.9913818]], dtype=float32)

In [0]:
test_sample = "진짜 개노잼이에요"
model.predict(tokenizer(test_sample))

array([[0.00396268]], dtype=float32)